# nn.Linear 层，
也常被称为`全连接层` (Fully Connected Layer) 或 `密集层` (`Dense Layer`)，是神经网络中最基本也是最常用的一种层。

- 核心功能：线性变换与特征组合 \
`nn.Linear` 层对输入数据执行一个线性变换 :  y = xW**T + b

- 其中 ：
    - x 是输入。
    - W 是该层的权重 (`weights`) 矩阵。
    - b 是该层的偏置 (`bias`) 向量。
    - y 是输出。
    - T转置

- 权重` W `和偏置` b `都是模型在训练过程中需要学习的参数。

- 它的主要作用是：

    - 特征组合：`nn.Linear` 层的每个输出神经元都与所有输入神经元相连接。这使得该层能够学习输入特征之间的全局关系，并将它们组合成更高级的表示。
    - 维度变换：它可以将输入数据从一个维度（特征空间）映射到另一个维度。你可以用它来增加或减少特征向量的长度。
    - 分类/回归头：在网络的末端，nn.Linear 层通常用作“决策头”。例如，在卷积神经网络（CNN）提取出图像的高级特征后，这些特征会被展平 (flatten) 并送入一个或多个 nn.Linear 层，最终输出每个类别的得分（用于分类）或一个连续值（用于回归）。

---
### 关键参数详解
nn.Linear 的构造函数非常简单：

In [ ]:
torch.nn.Linear(
    in_features,
    out_features,
    bias=True
)

### 1. in_features (整数)
含义：`输入样本的特征数量`（即输入`向量的长度`）。

- 如何设置：这个值必须与输入到该层的数据的最后一个维度的大小完全匹配。

例如，如果你的输入数据是一个包含784个像素值的展平图像，那么第一个 `nn.Linear` 层的 `in_features` 就必须是784。

如果它位于另一个 `nn.Linear` 层之后，它的 `in_features` 必须等于前一层的` out_features`。

### 2. out_features (整数)
- 含义：该层输出的特征数量（`即该层中神经元的数量`）。

    - 如何设置：这是你作为模型设计者需要定义的超参数。
    
    - 在隐藏层中，`out_features `的大小决定了模型的“宽度”和容量。

- 在输出层中，它由任务目标决定：

    - 多类别分类：`out_features` 等于类别总数 K。
    
    - 二元分类：`out_features` 通常为 1 (配合 sigmoid 激活函数)。
    
    - 回归：out_features 通常为 1 (如果预测单个值)。

### 3. bias (布尔值)
- 含义：是否在该层中添加一个可学习的偏置项 b。

- 如何设置：默认为 True。在绝大多数情况下，你都应该保持这个默认设置。偏置项增加了模型的灵活性，使其能够更好地拟合数据。

---
### 输入与输出的形状 (Shape)
这是 `nn.Linear` 层的一个非常重要的特性。

- 输入形状: (N, *, H_in)

    - N: 批次大小 (Batch Size)。
    
    - *: 表示任意数量的额外维度。
    
    - H_in: 输入特征数 (in_features)。nn.Linear 只对输入的最后一个维度进行操作。

- 输出形状: (N, *, H_out)
    
    - H_out: 输出特征数 (out_features)。

除了最后一个维度从` H_in `变为` H_out `之外，所有其他维度都保持不变。

---
- 示例：
    - 输入张量形状: (64, 784) (一个批次包含64个样本，每个样本有784个特征)
    
    - 线性层定义: `nn.Linear`(in_features=784, out_features=128)
    
    - 输出张量形状: (64, 128)
 
---

### 代码示例
示例1：构建一个简单的多层感知机 (MLP)\
这个例子展示了如何堆叠 `nn.Linear `层来处理表格数据或展平后的图像数据。

In [ ]:
import torch
import torch.nn as nn

# 定义一个用于MNIST分类的简单网络
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # MNIST图像是28x28=784
        # 第一个线性层：输入784，输出256
        self.flatten = nn.Flatten()  # 将输入维度展平
        self.fc1 = nn.Linear(in_features=784, out_features=256)
        self.relu1 = nn.ReLU()
        # 第二个线性层：输入256 (必须匹配上一层的输出)，输出128
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.relu2 = nn.ReLU()
        # 输出层：输入128，输出10 (对应0-9十个数字)
        self.fc3 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        # x的初始形状: (N, 1, 28, 28)
        # 首先将图像展平
        x = self.flatten(x)  # 自动处理维度  -> (N, 784)
        
        x = self.fc1(x)    # -> (N, 256)
        x = self.relu1(x)
        
        x = self.fc2(x)    # -> (N, 128)
        x = self.relu2(x)
        
        x = self.fc3(x)    # -> (N, 10)
        return x

# 创建模型实例
model = MLP()

# 创建一个假的输入数据 (批次大小为4)
dummy_input = torch.randn(4, 1, 28, 28)

# 前向传播
output = model(dummy_input)

print("输入形状:", dummy_input.shape)
print("展平后形状:", dummy_input.flatten(-1, 28*28).shape)
print("输出形状:", output.shape)

# 输出:
# 输入形状: torch.Size([4, 1, 28, 28])
# 展平后形状: torch.Size([4, 784])
# 输出形状: torch.Size([4, 10])

### 示例2：在CNN的末尾使用
这个例子展示了 nn.Linear 如何作为CNN的分类头。

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1), # -> (N, 16, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2) # -> (N, 16, 14, 14)
        )
        
        # 经过卷积和池化后，特征图大小为 16x14x14
        # 所以展平后的向量长度为 16 * 14 * 14 = 3136
        self.classifier = nn.Sequential(
            nn.Linear(in_features=16 * 14 * 14, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )

    def forward(self, x):
        # x的形状: (N, 1, 28, 28)
        x = self.conv_block(x) # -> (N, 16, 14, 14)
        
        # 展平特征图以送入Linear层
        x = torch.flatten(x, start_dim=1) # -> (N, 3136)
        
        output = self.classifier(x) # -> (N, 10)
        return output

# 创建模型和数据
model_cnn = CNN()
dummy_input = torch.randn(4, 1, 28, 28)
output = model_cnn(dummy_input)
print("\n--- CNN 示例 ---")
print("CNN输出形状:", output.shape)

# 输出:
# --- CNN 示例 ---
# CNN输出形状: torch.Size([4, 10])